In [ ]:
import pandas as pd
import win32print
import win32ui
import datetime as dt
import inflect
import requests
from io import StringIO
import re
from num2words import num2words
import textwrap
import numpy as np

day = dt.datetime.now().day
month = dt.datetime.now().month
year = dt.datetime.now().year

def convert_google_sheet_url(url):
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'
    new_url = re.sub(pattern, replacement, url)
    return new_url

In [ ]:
# ================= Carregamento da base ================= #
cpf_banco = pd.read_csv(convert_google_sheet_url('https://docs.google.com/spreadsheets/d/1R2ziBev9t4c8xJpWbf5rtzjFMpCgfEko7B3I2iBMhG4/edit?gid=0#gid=0'), dtype=str)

def consultar_nome_por_cpf(cpf_digitado):
    registro = cpf_banco.loc[cpf_banco['CPF'].astype(str) == str(cpf_digitado)]
    if registro.empty:
        return None
    return registro.iloc[0]['NOME']

produto_banco = pd.read_excel('estoque.xlsx')
produto_banco.columns = produto_banco.columns.str.strip()
produto_banco['PREÇO DE VENDA'] = np.around(produto_banco['PREÇO DE VENDA'].astype(float), 2)

def consultar_produto_por_nome(produto_digitado):
    dado = produto_banco.loc[produto_banco['PRODUTO'].astype(str) == str(produto_digitado)]
    if dado.empty:
        return None
    return dado.iloc[0]['NOME']

In [ ]:
cpf_banco

In [ ]:
produto_banco

In [ ]:
import tkinter as tk

# Lista a partir do dataframe
produtos_disponiveis = produto_banco['PRODUTO'].dropna().astype(str).tolist()
nomes_disponiveis = cpf_banco['CPF'].dropna().astype(str).tolist()

PRODUTO = None
VALOR = None
VENDEDOR = None
CPF = None
NOME = None




def atualizar_lista_cpf(*args):
    termo = entrada_cpf.get().lower()
    lista_cpf.delete(0, tk.END)

    for nome in nomes_disponiveis:
        if termo in nome.lower():
            lista_cpf.insert(tk.END, nome)


def selecionar_cpf(event):
    if lista_cpf.curselection():
        selecionado = lista_cpf.get(lista_cpf.curselection())
        entrada_cpf.delete(0, tk.END)
        entrada_cpf.insert(0, selecionado)

def atualizar_lista(*args):
    termo = entrada.get().lower()
    lista.delete(0, tk.END)

    for produto in produtos_disponiveis:
        if termo in produto.lower():
            lista.insert(tk.END, produto)

def selecionar_produto(event):
    if lista.curselection():
        selecionado = lista.get(lista.curselection())
        entrada.delete(0, tk.END)
        entrada.insert(0, selecionado)


def confirmar_produto():
    global PRODUTO, VALOR, QNTD, VENDEDOR, CPF, NOME
    
    produto_final = entrada.get()

    if produto_final in produtos_disponiveis:
        PRODUTO = produto_final

        linha = produto_banco[produto_banco['PRODUTO'] == PRODUTO]
        if not linha.empty:
            VALOR = float(linha['PREÇO DE VENDA'].values[0])

        QNTD = int(qntd_spinbox.get())
        VENDEDOR = vendedor_var.get()

        nome_final = entrada_cpf.get()

        linha_cpf = cpf_banco[cpf_banco['NOME'] == nome_final]
        if not linha_cpf.empty:
            CPF = linha_cpf['CPF'].values[0]
            NOME = nome_final
        else:
            print("Cliente inválido. Selecione a partir da lista.")
            return

        janela.destroy()
    else:
        print("Produto inválido. Selecione a partir da lista.")


# ---------- Interface ----------

janela = tk.Tk()
janela.title("Seleção inteligente de produto")
janela.geometry("400x500")

tk.Label(janela, text="Digite ou selecione o produto:").pack(pady=5)
tk.Label(janela, text="Quantidade:").pack(pady=2)

qntd_spinbox = tk.Spinbox(janela, from_=1, to=100, width=5)
qntd_spinbox.pack(pady=5)

tk.Label(janela, text="Vendedor:").pack(pady=2)
lista_vendedores = ["vendedor1", "vendedor2", "vendedor3"]

tk.Label(janela, text="Cliente (Nome / CPF):").pack(pady=2)

entrada_cpf = tk.Entry(janela, width=40)
entrada_cpf.pack(pady=5)
entrada_cpf.bind("<KeyRelease>", atualizar_lista_cpf)

lista_cpf = tk.Listbox(janela, width=100, height=6)
lista_cpf.pack(pady=5)
lista_cpf.bind("<<ListboxSelect>>", selecionar_cpf)

vendedor_var = tk.StringVar()
vendedor_var.set(lista_vendedores[0])

menu_vendedores = tk.OptionMenu(janela, vendedor_var, *lista_vendedores)
menu_vendedores.pack(pady=5)

entrada = tk.Entry(janela, width=40)
entrada.pack(pady=5)
entrada.bind("<KeyRelease>", atualizar_lista)

lista = tk.Listbox(janela, width=100, height=15)
lista.pack(pady=5)
lista.bind("<<ListboxSelect>>", selecionar_produto)

btn_confirmar = tk.Button(janela, text="Confirmar", command=confirmar_produto)
btn_confirmar.pack(pady=10)

janela.mainloop()


print(f"Produto: {PRODUTO}")
print(f"Valor: {VALOR*QNTD}")
print(f"Quantidade: {QNTD}")
print(f"Vendedor: {VENDEDOR}")
print(f"CPF: {CPF}")
print(f"Nome: {NOME}")


In [ ]:
valor_extenso = num2words(VALOR, lang='pt_BR', to='currency')

In [ ]:
texto = f"""
RECIBO FISCAL:

Declaro que recebi de {NOME}, 
no CPF sob o nº {CPF} a importância 
de R$ {VALOR*QNTD} 
({valor_extenso}), 
referente ao pagamento correspondente à 
aquisição do (s) seguinte (s) produto (s) 
no Bazar Beneficente de produtos doados 
pela Receita Federal:

Produto(s): {PRODUTO}
Quantidade: {QNTD}
Valor: R$ {VALOR}

Nova Serrana - MG, {day} de {month} de {year}.

Assinatura:

{VENDEDOR}

"""

print(texto)

In [ ]:

printer_name = win32print.GetDefaultPrinter()
hPrinter = win32print.OpenPrinter(printer_name)
hDC = win32ui.CreateDC()

hDC.CreatePrinterDC(printer_name)
hDC.StartDoc("Job")
hDC.StartPage()

font = win32ui.CreateFont({
    "name": "Consolas",   # fonte monoespaçada
    "height": 28,         # aumenta o tamanho vertical
    "width": 12,          # regula a largura horizontal
    "weight": 400,        # 400 = normal, 700 = bold
})

hDC.SelectObject(font)

texto = f"""
RECIBO FISCAL:

Declaro que recebi de {NOME}, 
no CPF sob o nº {CPF} a importância 
de R$ {VALOR} 
({valor_extenso}), 
referente ao pagamento correspondente à 
aquisição do (s) seguinte (s) produto (s) 
no Bazar Beneficente de produtos doados 
pela Receita Federal:

Produto(s): {PRODUTO}
Quantidade: {QNTD}
Valor: R$ {VALOR}

Nova Serrana - MG, {day} de {month} de {year}.

Assinatura:

{VENDEDOR}

"""

linhas_formatadas = []
for linha in texto.split("\n"):
    if linha.strip():
        linhas_formatadas.extend(textwrap.wrap(linha, width=LIMITE))
    else:
        linhas_formatadas.append("")

y = 10
for linha in linhas_formatadas:
    hDC.TextOut(10, y, linha)
    y += 40


hDC.EndPage()
hDC.EndDoc()
hDC.DeleteDC()